In [4]:
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv('../../dataset/train.csv')
test = pd.read_csv('../../dataset/test.csv')
# 前処理を一度にやるためにtrainとtestをconcatする
test['Transported'] = np.nan
train_test = pd.concat([train, test], axis=0, ignore_index=True, sort=False)

# split on `/` to cols (deck/num/side)
def split_cabin(df):
    cabin = df['Cabin'].str.split('/', expand=True).rename(columns={0: 'CabinDeck', 1: 'CabinNum', 2: 'CabinSide'})
    cabin['CabinNum'] = cabin['CabinNum'].astype(float)
    return pd.concat([df, cabin], axis=1)

# group passenger or not
def make_group(df):
    df['GroupId'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
    df['PeopleId'] = df['PassengerId'].apply(lambda x: x.split('_')[1])
    df['IsGroup'] = df['GroupId'].duplicated(keep=False)
    return df

# total room service, etc...
def total_bill(df):
    df['TotalBill'] = df[
        ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    return df

# binalize 3 classes
def binalize_bill(df, th1=5000, th2=20000): 
    df = total_bill(df)
    df['BillBins'] = df['TotalBill'].apply(
        lambda x: 0 if x < th1 else (2 if x > th1 and x < th2 else 3))
    return df


train_test = split_cabin(train_test)
train_test = make_group(train_test)
train_test = binalize_bill(train_test)

train_test.head()

/var/folders/zp/6qwnpvfn0cs2whczwk_5pvqh0000gs/T/ipykernel_37056/3830194740.py:5: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  train_test = pd.concat([train, test], axis=0, ignore_index=True, sort=False)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Name,Transported,CabinDeck,CabinNum,CabinSide,GroupId,PeopleId,IsGroup,TotalBill,BillBins
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,Maham Ofracculy,0.0,B,0.0,P,0001,01,False,0.0,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,Juanna Vines,1.0,F,0.0,S,0002,01,False,736.0,0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,Altark Susent,0.0,A,0.0,S,0003,01,True,10383.0,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,...,Solam Susent,0.0,A,0.0,S,0003,02,True,5176.0,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,...,Willy Santantines,1.0,F,1.0,S,0004,01,False,1091.0,0


### 使う特徴量を選ぶ

In [6]:
train_test = train_test[['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'CabinDeck', 'CabinNum', 'CabinSide', 'IsGroup', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalBill', 'BillBins', 'Transported']]
train_test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,CabinDeck,CabinNum,CabinSide,IsGroup,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,TotalBill,BillBins,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,B,0.0,P,False,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,Earth,False,TRAPPIST-1e,24.0,False,F,0.0,S,False,109.0,9.0,25.0,549.0,44.0,736.0,0,1.0
2,Europa,False,TRAPPIST-1e,58.0,True,A,0.0,S,True,43.0,3576.0,0.0,6715.0,49.0,10383.0,2,0.0
3,Europa,False,TRAPPIST-1e,33.0,False,A,0.0,S,True,0.0,1283.0,371.0,3329.0,193.0,5176.0,2,0.0
4,Earth,False,TRAPPIST-1e,16.0,False,F,1.0,S,False,303.0,70.0,151.0,565.0,2.0,1091.0,0,1.0


### Encoding

In [7]:
# HomePlanet, Destination, CabinSideはlabel encoding
for col in ['HomePlanet', 'Destination', 'CabinDeck', 'CabinSide']:
    train_test[col] = pd.factorize(train_test[col])[0]

In [8]:
# boolをintへ
def bool2int(df):
    for col in df.columns:
        if df[col].dtype == bool:
            df[col] = df[col].astype(float)
        if df[col].dtype == 'object':
            df[col] = df[col].map({True: 1, False: 0})
    return df

train_test = bool2int(train_test)

In [9]:
train_test

,HomePlanet,CryoSleep,Destination,Age,VIP,CabinDeck,CabinNum,CabinSide,IsGroup,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,TotalBill,BillBins,Transported
0,0,0.0,0,39.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,1,0.0,0,24.0,0.0,1,0.0,1,0.0,109.0,9.0,25.0,549.0,44.0,736.0,0,1.0
2,0,0.0,0,58.0,1.0,2,0.0,1,1.0,43.0,3576.0,0.0,6715.0,49.0,10383.0,2,0.0
3,0,0.0,0,33.0,0.0,2,0.0,1,1.0,0.0,1283.0,371.0,3329.0,193.0,5176.0,2,0.0
4,1,0.0,0,16.0,0.0,1,1.0,1,0.0,303.0,70.0,151.0,565.0,2.0,1091.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,1,1.0,0,34.0,0.0,3,1496.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN
12966,1,0.0,0,42.0,0.0,-1,NaN,-1,0.0,0.0,847.0,17.0,10.0,144.0,1018.0,0,NaN
12967,2,1.0,2,NaN,0.0,5,296.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN
12968,0,0.0,-1,NaN,0.0,5,297.0,0,0.0,0.0,2680.0,0.0,0.0,523.0,3203.0,0,NaN


### モデリング

In [17]:
# import lightgbm as lgbm
import optuna.integration.lightgbm as lgbm

In [18]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'learning_rate': 0.1,
    'importance_type': 'gain',
}

### 学習

In [19]:
import os
import random
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import accuracy_score

In [20]:
def set_seed(seed=3407):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


SEED = 3407
set_seed(SEED)
params.update(dict(seed=SEED))

In [21]:
# DataFrameをnp.ndarrayに変換
trainval = train_test[~train_test['Transported'].isna()]
test = train_test[train_test['Transported'].isna()]
# inputとlabelに分離
x_trainval = trainval.drop('Transported', axis=1).values
y_trainval = trainval.Transported.values
x_test = test.drop('Transported', axis=1).values

In [22]:
x_trainval.shape, y_trainval.shape, x_test.shape

((8693, 16), (8693,), (4277, 16))

In [27]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

val_scores = []
models = []
best_params = []
for fold, (train_inds, val_inds) in enumerate(kf.split(x_trainval)):
    
    x_train, x_val = x_trainval[train_inds], x_trainval[val_inds]
    y_train, y_val = y_trainval[train_inds], y_trainval[val_inds]

    lgb_train = opt_lgb.Dataset(x_train, y_train)
    lgb_val = opt_lgb.Dataset(x_val, y_val, reference=lgb_train)
    
    lgb_results = {}
    model = opt_lgb.train(
                    params,                    # ハイパーパラメータをセット
                    lgb_train,              # 訓練データを訓練用にセット
                    valid_sets=[lgb_train, lgb_val], # 訓練データとテストデータをセット
                    valid_names=['Train', 'Val'],    # データセットの名前をそれぞれ設定
                    num_boost_round=100,              # 計算回数
                    early_stopping_rounds=50,         # アーリーストッピング設定
                    evals_result=lgb_results,
                    verbose_eval=0,
                    )  
    best_params.append(model.params)

    y_val_proba = model.predict(x_val, num_iteration=model.best_iteration)
    y_val_pred = np.where(y_val_proba < 0.5, 0, 1)
    score = accuracy_score(y_val, y_val_pred)
    print(f'fold {fold}/acc: {score}')
    val_scores.append(score)
    models.append(model)

cv_score = np.mean(val_scores)
print(f'CV score: {cv_score}')

[I 2022-09-14 22:32:19,689] A new study created in memory with name: no-name-1daed285-4c46-4331-9763-b871d0ba3aec
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/Users/yamada/.pyenv/versions/3.8.0/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yamada/.pyenv/versions/3.8.0/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yamada/.pyenv/versions/3.8.0/lib/python3.8/site-packag

[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] Number of positive: 3498, number of negative: 3456
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005977 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1894
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 16
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503020 -> initscore=0.012080
[LightGBM] [Info] Start training from score 0.012080
Training until validation scores don't improve for 50 rounds


feature_fraction, val_score: 0.395494:  14%|#4        | 1/7 [00:00<00:05,  1.10it/s]

Did not meet early stopping. Best iteration is:
[100]	Train's binary_logloss: 0.267827	Val's binary_logloss: 0.395494
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] Number of positive: 3498, number of negative: 3456
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001094 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1894
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 16
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503020 -> initscore=0.012080
[LightGBM] [Info] Start training from score 0.012080
Training until validation scores don't improve for 50 rounds


feature_fraction, val_score: 0.391110:  29%|##8       | 2/7 [00:01<00:04,  1.24it/s]

...

Did not meet early stopping. Best iteration is:
[100]	Train's binary_logloss: 0.253136	Val's binary_logloss: 0.396808
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] Number of positive: 3511, number of negative: 3444
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001731 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1895
[LightGBM] [Info] Number of data points in the train set: 6955, number of used features: 16
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504817 -> initscore=0.019267
[LightGBM] [Info] Start training from score 0.019267
Training until validation scores don't improve for 50 rounds


min_data_in_leaf, val_score: 0.395728:  80%|########  | 4/5 [00:03<00:00,  1.15it/s]

Did not meet early stopping. Best iteration is:
[100]	Train's binary_logloss: 0.248393	Val's binary_logloss: 0.395845
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] Number of positive: 3511, number of negative: 3444
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002085 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1895
[LightGBM] [Info] Number of data points in the train set: 6955, number of used features: 16
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504817 -> initscore=0.019267
[LightGBM] [Info] Start training from score 0.019267
Training until validation scores don't improve for 50 rounds


min_data_in_leaf, val_score: 0.395728: 100%|##########| 5/5 [00:04<00:00,  1.14it/s]

Did not meet early stopping. Best iteration is:
[100]	Train's binary_logloss: 0.255925	Val's binary_logloss: 0.397453
fold 4/acc: 0.8107019562715765
CV score: 0.8164042798031487


In [29]:
importance = pd.DataFrame()

for i in range(len(models)):
    df = pd.DataFrame(models[i].feature_importances_,
                      index=trainval.columns[:-1], columns=[f'model{i+1}'])
    df = df.sort_values(f'model{i+1}', ascending=False)
    importance = pd.concat([importance, df], axis=1)

importance

AttributeError: 'Booster' object has no attribute 'feature_importances_'

### submit用のcsv作成

cvごとの推論の単純平均

In [42]:
y_preds = []

for i in range(len(models)):
    predictor = models[i]
    y_pred = predictor.predict(x_test, num_iteration=model.best_iteration)
    y_preds.append(y_pred)
ensemble = np.where(np.mean(y_preds, axis=0) < 0.5, 0, 1)

In [43]:
test_ids = pd.read_csv('../../dataset/test.csv')['PassengerId']

df_submit = pd.DataFrame(ensemble, index=test_ids, columns=['Transported'])
df_submit.Transported = df_submit.Transported.astype(bool)

In [44]:
df_submit

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True
...,...
9266_02,True
9269_01,False
9271_01,True


In [33]:
df_submit.to_csv('submission/lgbm_trial5_optuna.csv')